In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience
import pims
import trackpy as tp
import math
from scipy.misc import imsave
from scipy.misc import imread
################################################
# Read frames which have been data-cleaned (wire_particle_tracking_datacleaning). 
################################################
BR = [[5.7149E-08,1.1882E-05,-2.4321E-03],[8.8831E-06,-5.3566E-07,-3.8744E-02],[-1.6469E-03,3.3774E-02,9.9867E-01]]
TR = [[-5.7704E-08,1.1998E-05,-2.1514E-03],[8.9694E-06,5.4087E-07,-3.9328E-02],[-1.7813E-03,3.3894E-02,9.9865E-01]]
TL = [[1.1040E-07,2.2954E-05,-4.1584E-03],[1.7160E-05,-1.0348E-06,6.8654E-02],[-3.4505E-03,-7.3661E-02,9.9490E-01]]
BL = [[-1.0175E-07,2.1155E-05,-4.2910E-03],[1.5815E-05,9.5369E-07,6.2907E-02],[-2.8931E-03,-6.8254E-02,9.9567E-01]]
F_10point = [[  8.34360564e-08,   1.89022714e-05,  -3.73041652e-03],[  1.94501963e-05,   1.19512678e-06,  -5.90813947e-02],[ -3.16737708e-03,   5.30589622e-02,   9.96830069e-01]]
F_big = [[  1.75345969e-07,   1.87353876e-05,  -3.69560210e-03], [  1.92508173e-05,   9.79548859e-07,  -5.72239187e-02],[ -3.15390413e-03,   5.12220927e-02,   9.97034661e-01]]
F = F_big
Matr = F
MatrixL = np.array(Matr)
MatrixR = MatrixL.transpose()
Linex = np.arange(0,384,1)
v0L = imread('./FrameL_sum.tif')
v0R = imread('./FrameR_sum.tif')
bk0L = imread('./FrameL0.tif')
bk0R = imread('./FrameR0.tif')

In [30]:
df = pd.read_csv("TrackPair_PID.csv")
df
#Pair=df[(df['FF Ave']<10)&(df['Mass Diff']<50)]
Pair=df[(df['FF Ave']<5)&(df['Mass Diff']<50)&(df['Frame Count']>1000)&(df['FF Small Count']>100)]
Pair
#Pair = Pair[Pair['Mass Sqrt']<30]
#Pa = Pair[['PIDL','PIDR']]
#Pa.to_csv('TrackPair_PID_Clean.csv')

,Unnamed: 0,PIDL,PIDR,LenL,LenR,Xmin L,Xmax L,Ymin L,Ymax L,Xmin R,Xmax R,Ymin R,Ymax R,Frame Count,Mass Diff,FF Ave,FF Small Count
0,0,0,0,3891,1344,83.176707,197.111831,4.000000,96.000000,64.922388,109.015123,9.922388,54.039698,1336,24.541053,4.436425,871
24,24,0,502,3891,1479,83.176707,197.111831,4.000000,96.000000,112.987013,129.933464,58.853896,74.064257,1479,28.663611,1.567098,1479
263,6,1,73,4551,3941,137.870229,250.040388,5.844262,118.124760,192.017456,235.190840,0.717557,81.057357,3941,39.155166,0.560787,3941
630,71,12,917,3722,1653,32.000000,165.038462,104.128609,119.000000,0.744737,81.922737,109.027642,118.954887,1353,33.808398,1.283096,1353
811,9,128,241,2753,2597,28.000000,131.961240,63.896437,79.111984,28.859155,77.056022,68.079572,78.115108,2554,27.884381,2.807921,2554
1232,22,176,446,3801,2863,0.718121,212.200535,62.138182,117.851190,0.729167,99.127610,73.229947,104.861915,2785,26.650840,2.640624,2768
1738,13,224,318,3344,2574,119.000000,222.161209,78.862805,109.872796,145.085179,165.000000,78.086637,108.000000,2529,25.594180,0.712508,2529
2036,16,243,339,2717,1270,235.000000,239.948630,107.000000,127.011091,199.911765,213.075949,103.000000,120.123789,1270,22.965483,0.925518,1270
2261,17,254,344,2726,3252,46.000000,150.019022,116.890232,120.043478,6.000000,102.070022,119.042510,124.857143,2715,30.727056,0.540970,2715
2415,1,3,1,3983,3176,135.000000,222.924303,51.000000,100.950863,152.000000,173.828866,50.838462,81.929268,3170,46.733645,0.751302,3170


In [3]:
tL = pd.read_csv('trackL_inv_clean.csv')
tL.head()

,frame,diag_remembered,diag_search_range,diag_subnet,diag_subnet_iterations,diag_subnet_size,ecc,ep,frame.1,mass,particle,raw_mass,signal,size,x,y,range
0,0,NaN,NaN,NaN,NaN,NaN,0.528409,0.0,0,58.738626,0.0,157.0,17.220561,0.840730,83.176707,4.000000,146.441840
1,0,NaN,NaN,NaN,NaN,NaN,0.368421,0.0,0,68.174550,1.0,162.0,18.871848,0.850402,137.871972,5.854671,158.710600
2,0,NaN,NaN,NaN,NaN,NaN,0.402778,0.0,0,23.825708,3.0,44.0,6.841045,0.844317,135.000000,51.000000,101.122558
3,0,NaN,NaN,NaN,NaN,NaN,0.344340,0.0,0,210.185204,4.0,542.0,60.154015,0.844870,45.995511,54.836139,85.768837
4,0,NaN,NaN,NaN,NaN,NaN,0.446429,0.0,0,38.215492,7.0,81.0,11.794905,0.831479,33.000000,72.000000,48.225201


In [4]:
tR = pd.read_csv('trackR3_frameless.csv')
tR.head()

,frame,x,y,mass,size,ecc,signal,raw_mass,ep,frame.1,particle,range
0,0,64.922388,9.922388,96.057346,0.839332,0.419492,28.387096,254.0,0.0,0,0.0,62.373923
1,0,152.000000,50.865248,40.430107,0.846352,0.306931,11.469534,95.0,0.0,0,1.0,37.988652
2,0,168.000000,53.110169,33.835125,0.838683,0.337349,10.035842,80.0,0.0,0,2.0,117.007934
3,0,98.824236,58.971616,262.652325,0.849480,0.411498,73.118278,668.0,0.0,0,3.0,35.538575
4,0,271.901754,93.028070,81.720429,0.858395,0.347619,21.505376,194.0,0.0,0,6.0,93.251889
